# 강화 학습 개념에 대한 매우 간단한 소개 (상세)

# 목차

- [1. 소개: 강화 학습이란 무엇인가?](#1-소개)
  - [1.1 비유: 시행착오를 통한 학습](#11-비유)
  - [1.2 핵심 구성 요소](#12-핵심-구성-요소)
    - [1.2.1 에이전트 (Agent)](#121-에이전트)
    - [1.2.2 환경 (Environment)](#122-환경)
    - [1.2.3 상태 (State, s)](#123-상태)
    - [1.2.4 행동 (Action, a)](#124-행동)
    - [1.2.5 보상 (Reward, r)](#125-보상)
    - [1.2.6 정책 (Policy, π)](#126-정책)
    - [1.2.7 에피소드 (Episode)](#127-에피소드)
    - [1.2.8 목표 (Goal)](#128-목표)
  - [1.3 에이전트-환경 상호작용 루프](#13-상호작용-루프)
- [2. 과제: 간단한 그리드 월드](#2-과제)
  - [2.1 환경 설정](#21-환경-설정)
  - [2.2 상태, 행동, 보상 정의](#22-정의)
- [3. 우리의 간단한 "학습" 에이전트: MemoryBot](#3-에이전트)
  - [3.1 에이전트 목표 vs. 실제 RL 목표](#31-에이전트-목표)
  - [3.2 에이전트의 메모리 구조](#32-에이전트-메모리)
  - [3.3 에이전트의 간단한 정책 (전략)](#33-에이전트-정책)
  - [3.4 에이전트의 "학습" 과정 (메모리 업데이트)](#34-에이전트-학습)
- [4. 시뮬레이션 환경 설정](#4-설정)
  - [4.1 라이브러리 임포트](#41-임포트)
  - [4.2 환경 클래스 구현](#42-환경-클래스)
  - [4.3 환경 인스턴스 생성](#43-환경-인스턴스-생성)
- [5. 간단한 에이전트 로직 구현](#5-구현)
  - [5.1 에이전트 메모리 초기화](#51-메모리-초기화)
  - [5.2 행동 선택 (즉시 보상 기반 엡실론 그리디)](#52-행동-선택)
  - [5.3 에이전트 메모리 업데이트](#53-메모리-업데이트)
- [6. 시뮬레이션 루프 실행](#6-시뮬레이션)
  - [6.1 시뮬레이션 파라미터](#61-파라미터)
  - [6.2 실행을 위한 초기화](#62-초기화)
  - [6.3 메인 시뮬레이션 루프 (상세 단계)](#63-메인-루프)
- [7. 결과 시각화](#7-시각화)
  - [7.1 기본 성능 지표](#71-기본-플롯)
    - [7.1.1 시간에 따른 에피소드 보상](#711-보상)
    - [7.1.2 시간에 따른 에피소드 길이](#712-길이)
    - [7.1.3 엡실론 감쇠](#713-엡실론)
  - [7.2 에이전트의 지식 및 행동 분석](#72-고급-플롯)
    - [7.2.1 상태 방문 빈도](#721-상태-방문)
    - [7.2.2 평균 즉시 보상 맵](#722-평균-보상-맵)
    - [7.2.3 도출된 탐욕 정책 시각화](#723-정책-시각화)
    - [7.2.4 행동 선택 빈도 (예시 상태)](#724-행동-빈도)
    - [7.2.5 평균 보상 수렴 (예시 상태-행동)](#725-보상-수렴)
- [8. 이 간단한 에이전트의 한계](#8-한계)
- [9. 결론 및 다음 단계](#9-결론)

## 1. 소개: 강화 학습이란 무엇인가?

강화 학습(Reinforcement Learning, RL)은 지능적인 **에이전트**가 특정 **목표**를 달성하기 위해 **환경** 내에서 최적의 결정 순서를 내리도록 훈련하는 매력적인 머신러닝 분야입니다. (레이블된 데이터가 있는) 지도 학습이나 (데이터에서 패턴을 찾는) 비지도 학습과 달리, RL 에이전트는 **시행착오**를 통해 학습합니다.

### 1.1 비유: 시행착오를 통한 학습

개를 앉도록 훈련시키는 방법을 생각해 보세요.
- 개는 **에이전트**입니다.
- 방과 당신의 명령은 **환경**을 구성합니다.
- 개가 서 있거나 앉아 있는 것은 개의 **상태**입니다.
- 개가 앉거나 서기를 선택하는 것은 개의 **행동**입니다.
- 당신이 "앉아"라고 말했을 때 개가 앉으면, 간식(긍정적 **보상**)을 줍니다.
- 앉지 않으면 보상을 주지 않거나 약한 부정적 신호(부정적 보상)를 줄 수 있습니다.
- 시간이 지남에 따라 개는 **정책**을 학습합니다: "주인이 '앉아'라고 말하면, '앉는' 행동이 보통 간식(좋은 보상)으로 이어지므로, 그렇게 해야겠다."

RL은 유사한 원리로 작동하지만, 종종 훨씬 더 복잡한 시나리오에서 적용됩니다.

### 1.2 핵심 구성 요소

주요 요소들을 공식적으로 정의해 봅시다:

#### 1.2.1 에이전트 (Agent)
우리가 훈련하는 개체입니다. 환경의 상태를 인식하고 어떤 행동을 취할지 결정합니다.
*(우리의 예시: 그리드 상에서 어디로 이동할지 결정하는 간단한 프로그램)*

#### 1.2.2 환경 (Environment)
에이전트가 상호작용하는 외부 시스템입니다. 규칙, 물리 법칙, 행동의 결과를 정의합니다.
*(우리의 예시: 그리드, 벽, 목표 위치, 그리고 행동이 에이전트의 위치를 어떻게 바꾸는지)*

#### 1.2.3 상태 (State, $s$)
특정 순간의 환경에 대한 완전한 설명입니다. 에이전트는 상태를 사용하여 다음 행동을 결정합니다.
*(우리의 예시: 에이전트의 `(행, 열)` 좌표)*

#### 1.2.4 행동 (Action, $a$)
에이전트가 할 수 있는 가능한 움직임 또는 결정 중 하나입니다.
*(우리의 예시: 위, 아래, 왼쪽 또는 오른쪽으로 이동)*

#### 1.2.5 보상 (Reward, $r$)
상태 $s$에서 행동 $a$를 수행하고 상태 $s'$로 전환된 *후에* 환경으로부터 받는 스칼라 피드백 신호입니다. 해당 상태에서 취한 행동의 즉각적인 바람직함을 나타냅니다.
*(우리의 예시: 이동 결과에 따라 +10, -1 또는 -0.1)*

#### 1.2.6 정책 (Policy, $\pi$)
에이전트의 전략입니다. 주어진 상태에서 에이전트가 행동을 선택하는 방식을 정의합니다. 결정적(상태에 대해 항상 동일한 행동 선택)이거나 확률적(확률에 따라 행동 선택)일 수 있습니다.
*(우리의 예시: 상태에서 어떤 행동이 좋은 즉시 보상을 주었는지 기억하는 것에 기반한 $\epsilon$-그리디 전략)*

#### 1.2.7 에피소드 (Episode)
시작 상태에서 종료 상태에 도달하거나 시간 제한을 초과할 때까지의 에이전트-환경 상호작용의 전체 시퀀스입니다.
*(우리의 예시: (0,0)에서 (9,9)까지 그리드를 탐색하는 전체 시도)*

#### 1.2.8 목표 (Goal)
궁극적인 목표는 일반적으로 **반환값(return)**, 즉 에피소드 또는 잠재적으로 무한한 기간 동안의 할인된 보상의 누적 합계를 최대화하는 것입니다. $\text{반환값} = \sum_{t=0}^{T} \gamma^t r_{t+1}$. 여기서 $\gamma$는 할인 인자(0에서 1 사이)로, 먼 미래의 보상보다 즉각적인 보상을 우선시합니다.
*(우리의 예시: 목표에 빠르게 도달하고 벽을 피함으로써 가능한 가장 높은 점수를 얻는 것)*

### 1.3 에이전트-환경 상호작용 루프
RL의 핵심 프로세스는 다음 주기를 따릅니다:
1. 에이전트는 환경으로부터 현재 상태 $s_t$를 관찰합니다.
2. $s_t$를 기반으로 에이전트는 정책 $\pi$에 따라 행동 $a_t$를 선택합니다.
3. 에이전트는 행동 $a_t$를 수행합니다.
4. 환경은 새로운 상태 $s_{t+1}$로 전환됩니다.
5. 환경은 에이전트에게 보상 $r_t$를 제공합니다.
6. 에이전트는 경험 $(s_t, a_t, r_t, s_{t+1})$을 사용하여 지식이나 정책을 업데이트합니다(이것이 "학습" 단계입니다).
7. 새로운 상태 $s_{t+1}$에서 루프가 반복됩니다.

## 2. 과제: 간단한 그리드 월드

### 2.1 환경 설정
10x10 그리드를 사용합니다. 에이전트는 `(0,0)`에서 시작하여 `(9,9)`에 도달해야 합니다. 이 간단한 설정은 상태, 행동 및 그 결과를 명확하게 볼 수 있게 해줍니다.

### 2.2 상태, 행동, 보상 정의
- **상태:** 에이전트의 `(행, 열)` 튜플.
- **행동:** 4개의 이산적 행동: 0 (위), 1 (아래), 2 (왼쪽), 3 (오른쪽).
- **전환:** 결정적입니다. `(r, c)`에서 '위' 행동을 취하면 `r=0`(벽)이 아닌 이상 `(r-1, c)`로 이동합니다.
- **보상:**
    - +10: 목표 `(9, 9)` 도달 시.
    - -1: 벽에 부딪힐 때 (그리드 밖으로 이동).
    - -0.1: 다른 모든 단계 수행 시.
- **에피소드 종료:** 목표 도달 또는 단계 제한 초과 시.

## 3. 우리의 간단한 "학습" 에이전트: MemoryBot

### 3.1 에이전트 목표 vs. 실제 RL 목표
진정한 RL 에이전트는 *누적 할인 보상*을 최대화하는 것을 목표로 합니다. 우리의 간단한 'MemoryBot' 에이전트는 훨씬 간단한 목표를 가집니다: 과거에 현재 상태에서 높은 *평균 즉시 보상*을 얻게 했던 행동을 선택하려고 시도합니다. 미래 보상이나 할인 개념이 없습니다. 이것은 기본 메모리와 진정한 가치 기반 학습 간의 차이를 보여줍니다.

### 3.2 에이전트의 메모리 구조
중첩된 딕셔너리를 사용합니다: `memory[상태][행동] -> 보상_리스트`.
- `상태`는 `(행, 열)` 튜플입니다.
- `행동`은 정수(0-3)입니다.
- `보상_리스트`는 해당 `상태`에서 해당 `행동`을 취한 *직후* 받은 모든 즉시 보상을 저장합니다.

### 3.3 에이전트의 간단한 정책 (전략)
평균 즉시 보상 기반 $\epsilon$-그리디:
1. 0과 1 사이의 난수 `rand`를 생성합니다.
2. `rand < epsilon`이면: 무작위 행동(0, 1, 2 또는 3)을 선택합니다 (탐험).
3. `rand >= epsilon`이면 (활용):
    a. 현재 상태 `s`에 대해 가능한 각 행동 `a`의 평균 즉시 보상을 계산합니다: `AvgR(s, a) = mean(memory[s][a])` (빈 리스트는 평균이 0 또는 매우 작은 음수 값을 갖는 것으로 처리).
    b. 가장 높은 `AvgR(s, a*)`를 가진 행동 `a*`를 찾습니다.
    c. 이 최적 행동들 중 하나를 무작위로 선택합니다 (동점 처리).

### 3.4 에이전트의 "학습" 과정 (메모리 업데이트)
이것은 단순히 기록 관리입니다:
- 상태 $s_t$에서 행동 $a_t$를 취하고 보상 $r_t$를 받은 후, 단순히 $r_t$를 리스트 `memory[s_t][a_t]`에 추가합니다.

## 4. 시뮬레이션 환경 설정

### 4.1 라이브러리 임포트
필요한 Python 라이브러리를 임포트합니다.

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np # 수치 연산 (예: 평균)을 위해
import matplotlib.pyplot as plt # 결과 플로팅을 위해
import random # 무작위 선택 (탐험, 동점 처리)을 위해
from collections import defaultdict # 메모리를 위한 중첩 딕셔너리 생성에 편리함
from typing import Tuple, Dict, List, DefaultDict, Any # 타입 힌팅을 위해

# 재현성을 위한 랜덤 시드 설정
seed: int = 42
random.seed(seed)
np.random.seed(seed)

# Jupyter Notebook용 인라인 플로팅 활성화
%matplotlib inline

print("라이브러리 임포트 및 시드 설정 완료.")

### 4.2 환경 클래스 구현
그리드 월드 환경 클래스를 정의합니다. 각 부분에 주석을 추가하여 설명합니다.

In [ ]:
class GridEnvironmentSimple:
    """ 
    RL 개념 시연을 위한 기본 그리드 월드 환경.
    상태는 (행, 열) 튜플입니다. 행동은 0-3 정수입니다.
    """
    def __init__(self, rows: int = 10, cols: int = 10) -> None:
        """ 그리드 환경을 초기화합니다. """
        self.rows: int = rows # 행의 수
        self.cols: int = cols # 열의 수
        self.start_state: Tuple[int, int] = (0, 0) # 에이전트는 왼쪽 상단에서 시작
        self.goal_state: Tuple[int, int] = (rows - 1, cols - 1) # 목표는 오른쪽 하단
        self.state: Tuple[int, int] = self.start_state # 에이전트의 현재 위치
        # 행동 매핑 정의: 0=위, 1=아래, 2=왼쪽, 3=오른쪽
        self.action_map: Dict[int, Tuple[int, int]] = {
            0: (-1, 0), 1: (1, 0), 2: (0, -1), 3: (0, 1)
        }
        self.actions: List[int] = list(self.action_map.keys()) # 가능한 행동 인덱스 리스트
        self.action_dim: int = len(self.actions) # 가능한 행동의 수
        print(f"[환경] {rows}x{cols} 그리드 초기화 완료. 시작: {self.start_state}, 목표: {self.goal_state}")

    def reset(self) -> Tuple[int, int]:
        """ 에이전트를 시작 상태로 재설정합니다. 초기 상태를 반환합니다. """
        self.state = self.start_state
        # print(f"[환경] 상태 재설정: {self.state}") # 디버그
        return self.state

    def step(self, action: int) -> Tuple[Tuple[int, int], float, bool]:
        """
        행동을 실행하고, 상태를 업데이트하며, 결과를 반환합니다.
        
        반환값:
            (다음_상태, 보상, 종료_여부)
        """
        # 이미 목표에 도달했다면, 에피소드는 효과적으로 종료됨 (현재 상태, 0 보상, True 반환)
        if self.state == self.goal_state:
            # print(f"[환경] 이미 목표 {self.state}에 도달함. 행동 없음.") # 디버그
            return self.state, 0.0, True
        
        # 행동 맵에서 상태 변화량 가져오기
        dr, dc = self.action_map[action]
        current_row, current_col = self.state
        
        # 잠재적 새 위치 계산
        next_row, next_col = current_row + dr, current_col + dc
        
        # 기본 보상은 스텝 비용
        reward: float = -0.1
        
        # 경계(벽) 확인
        if not (0 <= next_row < self.rows and 0 <= next_col < self.cols):
            # 벽에 부딪힘: 상태는 변하지 않고, 보상은 페널티
            next_state_tuple = self.state 
            reward = -1.0 
            # print(f"[환경] {self.state}에서 행동 {action} 시도 중 벽 충돌. 다음 상태: {next_state_tuple}, 보상: {reward}") # 디버그
        else:
            # 유효한 이동: 내부 상태 업데이트
            self.state = (next_row, next_col)
            next_state_tuple = self.state
            # print(f"[환경] {(current_row, current_col)}에서 행동 {action}을 통해 {self.state}로 이동. 보상: {reward}") # 디버그
        
        # 새 상태가 목표인지 확인
        done: bool = (self.state == self.goal_state)
        if done:
            reward = 10.0 # 목표 도달 보상 할당
            # print(f"[환경] 목표 도달! 상태: {self.state}. 보상: {reward}, 종료: {done}") # 디버그
            
        # 결과 반환: 새 상태, 보상, 종료 플래그
        return next_state_tuple, reward, done

    def get_action_space_size(self) -> int:
        """ 가능한 행동의 수를 반환합니다. """
        return self.action_dim

### 4.3 환경 인스턴스 생성
우리의 그리드 월드 인스턴스를 생성합니다.

In [ ]:
# 환경 인스턴스 생성
simple_env: GridEnvironmentSimple = GridEnvironmentSimple(rows=10, cols=10)
# 환경으로부터 가능한 행동의 수 가져오기
n_actions_simple: int = simple_env.get_action_space_size()

## 5. 간단한 에이전트 로직 구현

메모리, 행동 선택, 학습(메모리 업데이트)을 위한 함수를 구현합니다.

### 5.1 에이전트 메모리 초기화
편의를 위해 `defaultdict`를 사용합니다. `memory[state]`에 접근했을 때 존재하지 않으면 자동으로 새로운 `defaultdict(list)`를 생성합니다. `memory[state][action]`에 접근했을 때 존재하지 않으면 자동으로 빈 `list`를 생성합니다.

In [ ]:
# 에이전트 메모리: 즉시 보상 리스트를 저장합니다.
# 키: 상태 튜플 (r, c)
# 값: 또 다른 defaultdict. 키: 행동 인덱스 (0-3), 값: 보상 리스트 [float]
agent_memory: DefaultDict[Tuple[int, int], DefaultDict[int, List[float]]] = \
    defaultdict(lambda: defaultdict(list))

print("에이전트 메모리 초기화 완료.")
print(f"방문하지 않은 상태 (0,1), 행동 0에 대한 메모리 접근 예시: {agent_memory[(0,1)][0]}")

### 5.2 행동 선택 (즉시 보상 기반 엡실론 그리디)
이 함수는 에이전트의 정책 $\pi(a|s)$를 구현합니다.

In [ ]:
def choose_simple_action(state: Tuple[int, int], 
                           memory: DefaultDict[Tuple[int, int], DefaultDict[int, List[float]]], 
                           epsilon: float, 
                           n_actions: int) -> int:
    """
    메모리에 저장된 평균 즉시 보상을 기반으로 엡실론 그리디를 사용하여 행동을 선택합니다.
    """
    # 탐험 또는 활용 결정
    if random.random() < epsilon:
        # --- 탐험 ---
        action: int = random.randrange(n_actions) # 무작위 행동 인덱스 선택 (0, 1, 2, 또는 3)
        # print(f"  -> 탐험 중: 무작위 행동 = {action}") # 디버그
        return action
    else:
        # --- 활용 ---
        # 현재 상태에서 가능한 각 행동에 대해 기억된 평균 보상 계산
        avg_rewards_for_actions: List[float] = []
        state_action_memory: DefaultDict[int, List[float]] = memory[state]
        
        for a in range(n_actions):
            rewards_list: List[float] = state_action_memory[a]
            if not rewards_list: # 이 상태에서 이 행동에 대한 메모리가 없다면
                avg_reward: float = 0.0 # 기본값 할당 (음수일 수도 있음)
            else:
                avg_reward = np.mean(rewards_list) # 기억된 보상의 평균 계산
            avg_rewards_for_actions.append(avg_reward)
            
        # 모든 행동 중 가장 높은 평균 보상 찾기
        max_avg_reward: float = max(avg_rewards_for_actions)
        
        # 이 최대 평균 보상을 달성하는 모든 행동의 리스트 얻기
        best_actions: List[int] = [a for a, avg_r in enumerate(avg_rewards_for_actions) if avg_r == max_avg_reward]
        
        # 최적 행동들 중 하나를 무작위로 선택 (동점 처리)
        action = random.choice(best_actions)
        # print(f"  -> 활용 중: 평균 보상={avg_rewards_for_actions}, 최적 행동={best_actions}, 선택됨={action}") # 디버그
        return action

### 5.3 에이전트 메모리 업데이트
이 함수는 에이전트의 간단한 "학습" 과정을 나타냅니다.

In [ ]:
def update_simple_memory(memory: DefaultDict[Tuple[int, int], DefaultDict[int, List[float]]], 
                           state: Tuple[int, int], 
                           action: int, 
                           reward: float) -> None:
    """
    받은 즉시 보상을 추가하여 에이전트의 메모리를 업데이트합니다.
    """
    # 특정 상태-행동 쌍에 대한 보상 리스트에 접근하여 새 보상 추가
    memory[state][action].append(reward)
    # print(f"  -> 메모리 업데이트됨: 상태={state}, 행동={action}, 추가된 보상={reward:.1f}, 행동에 대한 새 메모리: {memory[state][action]}") # 디버그

# 6. 시뮬레이션 루프 실행

여러 에피소드에 걸쳐 에이전트가 환경과 상호작용하는 것을 시뮬레이션합니다.

### 6.1 시뮬레이션 파라미터

In [ ]:
# 시뮬레이션 하이퍼파라미터
NUM_EPISODES: int = 1000 # 총 에피소드 수
MAX_STEPS_PER_EPISODE: int = 150 # 에피소드 당 최대 스텝 수
EPSILON_START_SIM: float = 1.0  # 완전히 무작위 탐험으로 시작
EPSILON_END_SIM: float = 0.01   # 작은 탐험 확률 유지
EPSILON_DECAY_SIM: float = 0.99 # 에피소드 당 감쇠율 (예: 0.99는 엡실론이 이전 값의 99%가 됨을 의미)
PRINT_EVERY_N_EPISODES: int = 100 # 진행 상황 출력 빈도

### 6.2 실행을 위한 초기화

In [ ]:
# 시뮬레이션 실행을 위한 환경 인스턴스 생성
env_run: GridEnvironmentSimple = GridEnvironmentSimple(rows=10, cols=10)
n_actions_run: int = env_run.get_action_space_size()

# 이번 실행을 위한 에이전트 메모리 초기화
agent_memory_run: DefaultDict[Tuple[int, int], DefaultDict[int, List[float]]] = \
    defaultdict(lambda: defaultdict(list))

# 플로팅을 위한 결과 저장 리스트 초기화
simple_episode_rewards: List[float] = []
simple_episode_lengths: List[int] = []
simple_episode_epsilons: List[float] = []

# 이번 실행을 위한 탐험률 초기화
current_epsilon_run: float = EPSILON_START_SIM

print("시뮬레이션 설정 완료.")

### 6.3 메인 시뮬레이션 루프 (상세 단계)
에이전트가 에피소드별로 환경과 상호작용합니다.

In [ ]:
print(f"간단한 에이전트 시뮬레이션을 {NUM_EPISODES} 에피소드 동안 시작합니다...")

# 지정된 에피소드 수만큼 반복
for i_episode in range(1, NUM_EPISODES + 1):
    
    # --- 에피소드 시작 --- 
    # 환경을 리셋하여 시작 상태 얻기
    current_state: Tuple[int, int] = env_run.reset()
    # 에피소드별 추적 변수 리셋
    current_episode_reward: float = 0.0
    
    # print(f"\n--- 에피소드 {i_episode} 시작 | 엡실론: {current_epsilon_run:.3f} ---") # 디버그

    # 에피소드 내 각 스텝에 대해 최대 허용 스텝까지 반복
    for t in range(MAX_STEPS_PER_EPISODE):
        
        # print(f" 스텝 {t+1}: 상태={current_state}") # 디버그
        
        # 1. 에이전트가 행동 선택
        action: int = choose_simple_action(
            current_state, 
            agent_memory_run, 
            current_epsilon_run, 
            n_actions_run
        )
        
        # 2. 에이전트가 환경에서 행동 수행
        next_state, reward, done = env_run.step(action)
        # print(f"   행동={action}, 보상={reward:.1f}, 다음 상태={next_state}, 종료={done}") # 디버그
        
        # 3. 에이전트가 메모리 업데이트 (즉시 보상으로부터 학습)
        update_simple_memory(agent_memory_run, current_state, action, reward)
        
        # 다음 타임 스텝을 위해 상태 업데이트
        current_state = next_state
        # 받은 보상을 에피소드의 총 보상에 더함
        current_episode_reward += reward
        
        # 에피소드가 종료되었는지 확인 (목표 도달 또는 다른 조건)
        if done:
            # print(f" 에피소드 {i_episode}가 스텝 {t+1}에서 완료됨. 목표 도달.") # 디버그
            break # 현재 에피소드 중단
            
    # --- 에피소드 종료 --- 
    # 루프가 MAX_STEPS에 도달하여 종료된 경우
    # if not done:
        # print(f" 에피소드 {i_episode}가 스텝 {t+1}에서 완료됨. 최대 스텝 도달.") # 디버그

    # 이번 에피소드의 결과 저장
    simple_episode_rewards.append(current_episode_reward)
    simple_episode_lengths.append(t + 1) # t는 마지막 스텝 인덱스이므로 길이는 t+1
    simple_episode_epsilons.append(current_epsilon_run)
    
    # 다음 에피소드를 위해 엡실론 감쇠
    current_epsilon_run = max(EPSILON_END_SIM, current_epsilon_run * EPSILON_DECAY_SIM)
    
    # 주기적으로 진행 상황 요약 출력
    if i_episode % PRINT_EVERY_N_EPISODES == 0:
        avg_reward = np.mean(simple_episode_rewards[-PRINT_EVERY_N_EPISODES:])
        avg_length = np.mean(simple_episode_lengths[-PRINT_EVERY_N_EPISODES:])
        print(f"에피소드 {i_episode}/{NUM_EPISODES} | 평균 보상 (최근 {PRINT_EVERY_N_EPISODES}): {avg_reward:.2f} | 평균 길이: {avg_length:.1f} | 엡실론: {current_epsilon_run:.3f}")

print(f"\n간단한 에이전트 시뮬레이션이 {NUM_EPISODES} 에피소드 후 종료되었습니다.")

# 7. 결과 시각화

플로팅은 에이전트의 성능이 시간에 따라 어떻게 변했는지 이해하는 데 도움이 됩니다.

### 7.1 기본 성능 지표

#### 7.1.1 시간에 따른 에피소드 보상

In [ ]:
# 에피소드 보상 플로팅
plt.figure(figsize=(20, 3))
# 한글 폰트 설정 (예: 나눔고딕)
plt.rcParams['font.family'] ='Malgun Gothic' # Windows
# plt.rcParams['font.family'] ='AppleGothic' # Mac
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 깨짐 방지

plt.plot(simple_episode_rewards, label='에피소드별 보상', alpha=0.7)
plt.title('간단한 에이전트: 에피소드별 총 보상')
plt.xlabel('에피소드')
plt.ylabel('총 보상')
plt.grid(True)

# 이동 평균 계산 및 플로팅
window_size = 50
if len(simple_episode_rewards) >= window_size:
    # numpy의 convolve를 사용하여 이동 평균 계산
    rewards_ma = np.convolve(simple_episode_rewards, np.ones(window_size)/window_size, mode='valid')
    # 이동 평균 플롯을 위한 x축 조정
    plt.plot(np.arange(len(rewards_ma)) + window_size - 1, rewards_ma, 
             label=f'{window_size}-에피소드 이동 평균', color='orange', linewidth=2)
    
plt.legend()
plt.show()

**해석:** 이 플롯은 각 에피소드에서 에이전트가 수집한 총 보상을 보여줍니다. 탐험과 무작위성 때문에 잡음이 많은 선이 예상됩니다. 주황색 선(이동 평균)은 이를 부드럽게 하여 일반적인 추세를 보여줍니다. 에이전트가 효과적으로 학습하고 있다면(간단한 전략임에도 불구하고), 이동 평균이 상승하는 경향을 보일 것으로 예상됩니다. 이는 시간이 지남에 따라 더 높은 점수를 얻고 있음(목표에 더 자주 또는 더 빨리 도달하거나 벽을 더 잘 피함)을 나타냅니다.

#### 7.1.2 시간에 따른 에피소드 길이

In [ ]:
# 에피소드 길이 플로팅
plt.figure(figsize=(20, 3))
plt.plot(simple_episode_lengths, label='에피소드별 스텝 수', alpha=0.7)
plt.title('간단한 에이전트: 에피소드별 스텝 수')
plt.xlabel('에피소드')
plt.ylabel('스텝 수')
plt.grid(True)

# 이동 평균 계산 및 플로팅
window_size = 50
if len(simple_episode_lengths) >= window_size:
    lengths_ma = np.convolve(simple_episode_lengths, np.ones(window_size)/window_size, mode='valid')
    plt.plot(np.arange(len(lengths_ma)) + window_size - 1, lengths_ma, 
             label=f'{window_size}-에피소드 이동 평균', color='orange', linewidth=2)
    plt.legend()

plt.show()

**해석:** 이 플롯은 각 에피소드를 완료하는 데 에이전트가 몇 스텝을 걸렸는지 보여줍니다. 에이전트가 목표에 더 효율적으로 도달하는 법을 배우면 스텝 수가 시간이 지남에 따라 감소해야 합니다. 이동 평균의 하향 추세는 효율성 향상을 나타냅니다.

#### 7.1.3 엡실론 감쇠

In [ ]:
# 엡실론 감쇠 플로팅
plt.figure(figsize=(20, 3))
plt.plot(simple_episode_epsilons)
plt.title('간단한 에이전트: 에피소드에 따른 엡실론 감쇠')
plt.xlabel('에피소드')
plt.ylabel('엡실론 값')
plt.grid(True)
plt.show()

**해석:** 이 플롯은 탐험률($\epsilon$)이 우리가 정의한 감쇠 일정에 따라 에피소드 동안 어떻게 감소했는지 확인시켜 줍니다. 처음에는 높게 시작하여(탐험 장려) 점차 낮아집니다(학습된 지식 활용 장려).

### 7.2 에이전트의 지식 및 행동 분석

에이전트가 무엇을 배웠고 어떻게 행동하는지 더 자세히 살펴보겠습니다.

#### 7.2.1 상태 방문 빈도
이는 에이전트가 그리드의 어느 부분을 가장 많이 탐험했는지 보여줍니다.

In [ ]:
def plot_state_visitation(memory: DefaultDict[Tuple[int, int], DefaultDict[int, List[float]]], 
                            env: GridEnvironmentSimple) -> None:
    """ 상태 방문 횟수의 히트맵을 플로팅합니다. """
    rows = env.rows
    cols = env.cols
    visit_counts = np.zeros((rows, cols))

    print("상태 방문 횟수 계산 중...")
    for state, action_dict in memory.items():
        r, c = state
        total_visits_to_state = sum(len(rewards) for rewards in action_dict.values())
        if 0 <= r < rows and 0 <= c < cols:
             visit_counts[r, c] = total_visits_to_state

    fig, ax = plt.subplots(figsize=(cols * 0.8, rows * 0.8))
    # 방문 횟수가 매우 다양하면 로그 스케일 사용, 아니면 선형 스케일 사용
    if np.max(visit_counts) > 10 * np.median(visit_counts[visit_counts > 0]): 
         im = ax.imshow(visit_counts + 1e-9, cmap='hot', origin='lower', norm=plt.cm.colors.LogNorm())
         title = "상태 방문 횟수 (로그 스케일)"
    else:
         im = ax.imshow(visit_counts, cmap='hot', origin='lower')
         title = "상태 방문 횟수 (선형 스케일)"

    # 텍스트 주석 추가 (선택 사항, 복잡해질 수 있음)
    # for r in range(rows):
    #     for c in range(cols):
    #         ax.text(c, r, f"{int(visit_counts[r, c])}", ha="center", va="center", color="w" if visit_counts[r,c] > np.max(visit_counts)/2 else "black")

    start_r, start_c = env.start_state
    goal_r, goal_c = env.goal_state
    ax.text(start_c, start_r, 'S', ha='center', va='center', color='cyan', weight='bold')
    ax.text(goal_c, goal_r, 'G', ha='center', va='center', color='lime', weight='bold')
    
    ax.set_xticks(np.arange(cols))
    ax.set_yticks(np.arange(rows))
    ax.set_xticklabels(np.arange(cols))
    ax.set_yticklabels(np.arange(rows))
    ax.set_xticks(np.arange(-.5, cols, 1), minor=True)
    ax.set_yticks(np.arange(-.5, rows, 1), minor=True)
    ax.grid(which='minor', color='black', linestyle='-', linewidth=0.5)
    ax.set_title(title)
    fig.colorbar(im)
    plt.show()

# 상태 방문 플로팅
plot_state_visitation(agent_memory_run, env_run)

**해석:** 이 히트맵은 훈련 중 에이전트가 어떤 그리드 셀을 가장 자주 방문했는지 보여줍니다. 밝은 색상은 더 많은 방문을 나타냅니다. 에이전트가 탐험하는 경향이 있는 경로를 볼 수 있습니다. 이상적으로는 특히 초기에 상당한 탐험을 보여야 하며, 훈련 후반에는 목표로 이어지는 경로를 따라 더 높은 집중도를 보일 수 있습니다.

#### 7.2.2 평균 즉시 보상 맵
이는 각 상태에서 최상의 *즉각적인* 결과에 대한 에이전트의 학습된 추정치를 보여줍니다.

In [ ]:
def plot_average_rewards_grid(memory: DefaultDict[Tuple[int, int], DefaultDict[int, List[float]]], 
                              env: GridEnvironmentSimple) -> None:
    """ 각 상태에서 최적 행동에 대해 학습된 평균 즉시 보상을 플로팅합니다. """
    rows = env.rows
    cols = env.cols
    avg_reward_grid = np.full((rows, cols), -np.inf) # -inf로 초기화
    
    print("시각화를 위한 평균 보상 계산 중...")
    for r in range(rows):
        for c in range(cols):
            state = (r, c)
            if state == env.goal_state:
                # 목표 보상 사용 (예: 10)
                # 목표 상태에서 어떤 행동(예: 0)을 해도 종료되므로, step 함수의 보상을 사용
                _next_state, reward, _done = env.step(0) 
                avg_reward_grid[r, c] = reward 
                continue
                
            action_values = []
            state_memory = memory[state]
            has_experience = False
            for a in range(env.get_action_space_size()):
                rewards = state_memory[a]
                if rewards:
                     has_experience = True
                     avg_reward = np.mean(rewards)
                     action_values.append(avg_reward)
                else:
                     # 시도되지 않은 행동 표시
                     action_values.append(-np.inf) 
                     
            if not has_experience:
                 # 방문하지 않은 상태에 대해 구분되는 낮은 값 할당
                 avg_reward_grid[r, c] = -5 
            else:
                 best_avg_reward = max(action_values)
                 if best_avg_reward == -np.inf:
                     # 상태는 방문했지만, 어떤 행동도 시도되지 않은 이상한 경우 (이론상 발생 안함)
                     avg_reward_grid[r, c] = -5 # 방문하지 않은 것으로 처리
                 else:
                      avg_reward_grid[r, c] = best_avg_reward

    # viridis 컬러맵 사용, 5개의 구분된 색상
    cmap = plt.cm.get_cmap('viridis', 5)  
    # 색상 경계 정의 (-5: 미방문, -1: 벽 근처, -0.1: 일반 스텝, >0: 목표 근처, 10: 목표)
    bounds = [-6, -1.5, -0.5, 0, 0.1, 11] 
    norm = plt.cm.colors.BoundaryNorm(bounds, cmap.N)

    fig, ax = plt.subplots(figsize=(cols * 0.8, rows * 0.8))
    im = ax.imshow(avg_reward_grid, cmap=cmap, norm=norm, origin='lower')

    # 텍스트 주석 추가
    for r in range(rows):
        for c in range(cols):
            val = avg_reward_grid[r, c]
            text_val = f"{val:.1f}" if val > -np.inf and val != -5 else ("N/A" if val == -5 else "?")
            color = "white" if abs(val) > 1 and val != -5 else "black"
            if val == -5: color = "grey"
            ax.text(c, r, text_val, ha="center", va="center", color=color, fontsize=7)
                           
    start_r, start_c = env.start_state
    goal_r, goal_c = env.goal_state
    ax.text(start_c, start_r, 'S', ha='center', va='center', color='red', weight='bold')
    ax.text(goal_c, goal_r, 'G', ha='center', va='center', color='white', weight='bold')
    
    ax.set_xticks(np.arange(cols))
    ax.set_yticks(np.arange(rows))
    ax.set_xticklabels(np.arange(cols))
    ax.set_yticklabels(np.arange(rows))
    ax.set_xticks(np.arange(-.5, cols, 1), minor=True)
    ax.set_yticks(np.arange(-.5, rows, 1), minor=True)
    ax.grid(which='minor', color='black', linestyle='-', linewidth=0.5)
    ax.set_title("간단한 에이전트: 최적 행동의 평균 즉시 보상")
    
    # 컬러바 틱 수정 - 경계 값의 중간 지점 사용
    tick_labels = ['미방문', '벽충돌', '일반', '목표근처', '목표']
    ticks = [(bounds[i] + bounds[i+1]) / 2 for i in range(len(bounds)-1)]
    cbar = fig.colorbar(im, ticks=ticks)
    cbar.ax.set_yticklabels(tick_labels)
    plt.show()

# 학습된 평균 보상 플로팅
plot_average_rewards_grid(agent_memory_run, env_run)

**해석:** 이 히트맵은 해당 상태에서 경험한 최상의 즉시 보상*만을* 기준으로 에이전트가 학습한 상태 선호도를 보여줍니다. 높은 값(밝은 색상, +10 근처)은 목표 지점에서 나타나야 합니다. 벽에 인접한 상태는 에이전트가 자주 부딪혔다면 더 낮은 값(어두운 색상, -1 근처)을 보일 수 있습니다. 벽이나 목표에서 멀리 떨어진 상태는 스텝 비용(-0.1) 근처에 모일 수 있습니다. 'N/A' 또는 뚜렷하게 낮은 값은 에이전트가 해당 상태에서 행동을 선택한 경험이 전혀 없거나 거의 없음을 나타냅니다.

#### 7.2.3 도출된 탐욕 정책 시각화
이는 에이전트가 메모리를 기반으로 탐욕적으로(epsilon=0) 행동할 경우 각 상태에서 어떤 행동을 취할지 보여줍니다.

In [ ]:
def plot_simple_policy_grid(memory: DefaultDict[Tuple[int, int], DefaultDict[int, List[float]]], 
                              env: GridEnvironmentSimple) -> None:
    """ 에이전트의 메모리에서 파생된 탐욕 정책을 플로팅합니다. """
    rows = env.rows
    cols = env.cols
    policy_grid: np.ndarray = np.empty((rows, cols), dtype=str)
    # 행동 기호: 0: 위, 1: 아래, 2: 왼쪽, 3: 오른쪽
    action_symbols: Dict[int, str] = {0: '↑', 1: '↓', 2: '←', 3: '→'} 

    fig, ax = plt.subplots(figsize=(cols * 0.7, rows * 0.7)) # 크기 조정

    print("메모리로부터 탐욕 정책 계산 중...")
    for r in range(rows):
        for c in range(cols):
            state = (r, c)
            # 목표 상태 표시
            if state == env.goal_state:
                symbol = 'G'
                color = 'green'
            else:
                # 메모리 기반 탐욕 행동 얻기 (epsilon=0.0)
                best_action = choose_simple_action(state, memory, epsilon=0.0, n_actions=env.action_dim)
                
                # 상태에 기록된 경험이 있는지 확인
                state_memory = memory[state]
                has_experience = any(state_memory[a] for a in range(env.action_dim))
                
                if not has_experience:
                    # 방문하지 않았거나 탐험되지 않은 상태 표시
                    symbol = '.' 
                    color = 'grey'
                else:
                    symbol = action_symbols[best_action]
                    color = 'black'
                
            policy_grid[r, c] = symbol
            ax.text(c, r, symbol, ha='center', va='center', color=color, fontsize=10, 
                    weight='bold' if symbol == 'G' else 'normal')

    # 플롯 서식 지정
    ax.matshow(np.zeros((rows, cols)), cmap='Greys', alpha=0.1) # 밝은 배경 그리드
    ax.set_xticks(np.arange(-.5, cols, 1), minor=True)
    ax.set_yticks(np.arange(-.5, rows, 1), minor=True)
    ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
    ax.set_xticks([]) # 축 눈금 숨기기
    ax.set_yticks([])
    ax.set_title("간단한 에이전트: 도출된 탐욕 정책 (평균 즉시 보상 기반)")
    plt.show()

# 도출된 정책 플로팅
plot_simple_policy_grid(agent_memory_run, env_run)

**해석:** 이것은 에이전트가 충분히 탐험한 각 상태에서 즉각적으로 최선이라고 *생각하는* 방향을 보여줍니다. 화살표는 선호하는 행동을 나타냅니다. '.'는 에이전트가 메모리가 없거나 행동을 시도하지 않은 상태를 나타냅니다. 'G'는 목표입니다. 벽에서 멀어지는 것과 같은 패턴을 볼 수 있지만, 에이전트가 장기적인 결과를 고려하지 않기 때문에 목표로 가는 일관된 경로는 나타나지 않을 가능성이 높습니다.

#### 7.2.4 행동 선택 빈도 (예시 상태)
시작 상태 `(0,0)`에서 각 행동이 얼마나 자주 선택되었는지 봅시다.

In [ ]:
def plot_action_frequency(memory: DefaultDict[Tuple[int, int], DefaultDict[int, List[float]]], 
                          state_to_analyze: Tuple[int, int],
                          n_actions: int) -> None:
    """ 특정 상태에서 각 행동이 얼마나 자주 취해졌는지 막대 차트로 플로팅합니다. """
    action_counts = [len(memory[state_to_analyze][a]) for a in range(n_actions)]
    action_labels = ['위', '아래', '왼쪽', '오른쪽'] # 0=위, 1=아래, 2=왼쪽, 3=오른쪽 가정
    
    plt.figure(figsize=(6, 4))
    plt.bar(action_labels, action_counts, color=['lightblue', 'lightcoral', 'lightsalmon', 'lightgreen'])
    plt.title(f'상태 {state_to_analyze}에서의 행동 빈도')
    plt.xlabel('행동')
    plt.ylabel('선택된 횟수')
    plt.grid(axis='y', linestyle='--')
    plt.show()
    
# 시작 상태에 대한 행동 빈도 플로팅
start_state = env_run.start_state
plot_action_frequency(agent_memory_run, start_state, n_actions_run)

**해석:** 이 막대 차트는 에이전트가 지정된 상태(여기서는 시작 상태)에 있을 때 각 행동이 선택된 횟수를 보여줍니다. 훈련 초기(높은 엡실론)에는 무작위 탐험으로 인해 횟수가 비슷할 수 있습니다. 나중에 에이전트가 즉시 보상을 기반으로 선호도를 개발했다면 일부 막대가 다른 막대보다 훨씬 높을 수 있습니다.

#### 7.2.5 평균 보상 수렴 (예시 상태-행동)
특정 상태-행동 쌍에 대한 평균 즉시 보상에 대한 에이전트의 추정치가 시간이 지남에 따라 어떻게 변했는지 추적해 봅시다.

In [ ]:
def plot_reward_convergence(memory_history: List[DefaultDict[Tuple[int, int], DefaultDict[int, List[float]]]],
                            state_to_analyze: Tuple[int, int],
                            action_to_analyze: int) -> None:
    """ 특정 상태-행동 쌍에 대한 실행 평균 보상을 에피소드에 걸쳐 플로팅합니다. """
    
    avg_rewards_over_time = []
    total_rewards_so_far = 0.0
    count = 0
    
    # 이 플롯을 위해서는 훈련 중에 메모리 스냅샷을 다시 실행하거나 저장해야 합니다.
    # 최종 메모리에서 재계산하여 근사화해 봅시다 (시간 경과에 따른 정확도는 떨어짐).
    # 더 나은 접근 방식은 훈련 중에 스냅샷을 저장하거나 실행 평균을 재계산하는 것입니다.
    
    # --- 최종 메모리를 사용한 근사 --- 
    # 이것은 최종 분포를 보여주며, 수렴 경로는 아닙니다
    # memory_history는 최종 메모리 하나만 가진 리스트로 가정합니다.
    if not memory_history:
        print(f"메모리 기록이 없습니다.")
        return
        
    final_memory = memory_history[-1]
    rewards_list = final_memory.get(state_to_analyze, {}).get(action_to_analyze, [])
    if not rewards_list:
        print(f"상태 {state_to_analyze}, 행동 {action_to_analyze}에 대해 기록된 데이터가 없습니다.")
        return
        
    running_averages = []
    current_sum = 0.0
    for i, reward in enumerate(rewards_list):
        current_sum += reward
        running_averages.append(current_sum / (i + 1))
    
    plt.figure(figsize=(10, 5))
    plt.plot(running_averages)
    plt.title(f'상태 {state_to_analyze}, 행동 {action_to_analyze}에 대한 즉시 보상의 실행 평균')
    plt.xlabel('해당 행동이 취해진 횟수')
    plt.ylabel('평균 즉시 보상')
    # 실제 평균 보상 값에 수평선 추가 (예: -0.1)
    # 실제 값 계산 필요: env.step를 사용하여 해당 상태, 행동의 즉시 보상 확인
    # env_temp = GridEnvironmentSimple(rows=env_run.rows, cols=env_run.cols) # 임시 환경
    # env_temp.state = state_to_analyze
    # _, expected_reward, _ = env_temp.step(action_to_analyze)
    # plt.axhline(y=expected_reward, color='r', linestyle='--', label=f'기대 보상 ({expected_reward:.1f})')
    plt.grid(True)
    # plt.legend()
    plt.show()

# 실제 수렴을 플로팅하려면 메모리 스냅샷이 필요합니다.
# 최종 메모리를 저장하면 추정치 분포를 플로팅할 수 있습니다.
example_state = (1, 1) 
example_action = 3 # 오른쪽
print(f"\n상태={example_state}, 행동={example_action}에 대한 보상 분석 중:")
# plot_reward_convergence 함수는 메모리 기록 리스트를 받도록 설계되었지만,
# 여기서는 최종 메모리 하나만 사용하여 실행 평균 추세를 보여줍니다.
plot_reward_convergence([agent_memory_run], example_state, example_action) 

# 추가로, 해당 쌍에 대해 받은 보상의 히스토그램을 플로팅합니다.
rewards_list_final = agent_memory_run.get(example_state, {}).get(example_action, [])
if rewards_list_final:
    plt.figure(figsize=(6, 4))
    plt.hist(rewards_list_final, bins=10, density=True, alpha=0.7, color='skyblue')
    # plt.hist(rewards_list_final, density=True, alpha=0.7, color='skyblue', align='left') # 이산값일 경우
    plt.title(f'상태 {example_state}, 행동 {example_action}에 대한 보상 히스토그램')
    plt.xlabel('즉시 보상')
    plt.ylabel('빈도')
    plt.grid(axis='y', linestyle='--')
    # 보상 값(-1, -0.1, 10)에 맞게 x축 눈금 설정 시도
    # unique_rewards = sorted(list(set(rewards_list_final)))
    # plt.xticks(unique_rewards)
    plt.show()
else:
     print(f"상태 {example_state}, 행동 {example_action}에 대해 기록된 데이터가 없습니다.")

**해석:** 선 플롯은 특정 상태((1,1))에서 특정 행동(오른쪽)을 취하는 것에 대한 평균 즉시 보상의 *추정치*가 해당 상황을 더 자주 경험함에 따라 어떻게 변하는지 보여줍니다. 이상적으로 이 평균은 해당 상태-행동 쌍에 대한 실제 기대 즉시 보상(이 경우 -0.1, 해당 상태에서 목표나 벽으로 직접 이어지지 않는 한)으로 수렴해야 합니다. 히스토그램은 해당 쌍에 대해 실제로 받은 보상의 분포를 보여줍니다. 우리의 결정론적 환경에서는 이상적으로 -0.1(또는 해당되는 경우 -1 또는 +10)에 단일 막대가 있어야 합니다.

## 8. 이 간단한 에이전트의 한계

기본적인 RL 상호작용 루프를 설명하는 이 MemoryBot은 주요 단점을 가지고 있습니다:

1.  **장기 계획 부재:** *다음 즉시 보상*만을 고려합니다. 지금은 나빠 보이지만(-0.1 보상) 나중에 목표(+10 보상)에 훨씬 빨리 도달할 수 있게 하는 움직임을 학습할 수 없습니다. 이것이 최적 경로를 찾는 것을 방해하는 근본적인 한계입니다.
2.  **테이블 형태 표현:** 모든 단일 상태-행동 쌍에 대한 데이터를 저장해야 합니다. 이는 크거나 연속적인 상태 공간을 가진 환경에는 확장되지 않습니다.
3.  **일반화 부재:** 상태 `(1,1)`에 대한 학습은 유사한 상태 `(1,2)`에 대한 정보를 제공하지 않습니다.
4.  **비효율적인 메모리:** 받은 *모든* 보상을 저장하는 것은 메모리 집약적일 수 있습니다. Q-러닝과 같은 알고리즘은 실행 평균 또는 학습률을 사용하여 점진적으로 추정치를 업데이트합니다.

## 9. 결론 및 다음 단계

이 상세한 노트북은 강화 학습의 기초 개념인 **에이전트-환경 상호작용 루프**, **상태**, **행동**, **보상**의 역할, 에이전트 **정책**의 아이디어, 그리고 **에피소드**를 통한 학습을 시연했습니다. 우리는 상태-행동 쌍과 관련된 *평균 즉시 보상*을 기억함으로써 순수하게 학습하는 매우 간단한 에이전트 'MemoryBot'을 구현했습니다.

전반적인 성능(보상, 길이), 탐험 패턴(상태 방문, 행동 빈도), 에이전트의 학습된 지식(평균 보상 맵, 수렴) 추적 등 다양한 시각화를 통해, 이 간단한 메모리 기반 적응조차도 기본적인 벽 회피와 같은 *일부* 행동 변화로 이어진다는 것을 관찰했습니다.

그러나 결정적인 한계는 **장기적인 누적 보상**을 고려할 수 없다는 점이며, 이는 진정으로 최적인 전략을 찾는 것을 방해합니다. 이는 더 정교한 RL 알고리즘의 필요성을 동기 부여합니다:

-   **Q-러닝 / 가치 반복:** 예상 *누적* 미래 보상($Q(s,a)$ 또는 $V(s)$)을 학습하여 장기 계획을 가능하게 합니다.
-   **심층 Q-네트워크 (DQN):** 신경망을 사용하여 $Q(s,a)$를 근사화하여 크거나 연속적인 상태 공간에 걸쳐 일반화를 허용합니다.
-   **정책 그래디언트 방법 (REINFORCE, A2C, PPO 등):** 파라미터화된 정책 $\pi(a|s)$를 직접 학습하며, 연속적인 행동 및 확률적 정책에 적합합니다.
-   **모델 기반 방법 (Dyna-Q, PlaNet):** 환경 모델을 학습하여 시뮬레이션을 통한 계획을 수행하며, 종종 샘플 효율성을 향상시킵니다.

이 기본적인 상호작용 루프와 보상으로부터 학습하는 개념을 이해하는 것은 이러한 더 강력한 RL 기법을 탐구하기 위한 필수적인 기초입니다.